In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

In [ ]:
def fewshot_prompt(context, question, game_elements, examples):
    """
    Creates a few-shot prompt for the model to generate Pygame-based interactive games.

    Parameters:
    - context: str, contextual information for the game development
    - question: str, specific game requirements or features to implement
    - game_elements: list, list of game elements/objects to include in the game
    - examples: list of dicts, each containing 'question', 'context', and 'output' as keys to provide examples
    Returns:
    - str, the formatted few-shot prompt
    """

    # Format the game elements
    elements = ", ".join(game_elements)

    # Format the examples for the prompt
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )

    # Construct the few-shot prompt
    prompt = f"""
    You are an experienced game developer with 15 years of experience creating interactive games using Pygame.
    Your task is to generate high-quality Python code for interactive games using Pygame
    based on the provided context and user question.

    Here are some examples of similar game development tasks you have completed successfully:
    {formatted_examples}

    Now, using these examples as a reference, generate code for the following game development task:

    Context: {context}
    User Question: {question}
    Include game elements: {elements}
    Ensure that:
    - The code is well-structured and follows Pygame best practices
    - The game is fully playable with proper collision detection and game mechanics
    - Include proper game loop, event handling, and rendering
    - Output only the Python code with necessary imports
    - The game should be interactive and engaging
    """
    return prompt

In [ ]:
def get_answer_using_fewshot(context, question, game_elements, examples):
    """
    Generates the response from the model based on a few-shot prompt for game development.

    Parameters:
    - context: str, contextual information for the game development
    - question: str, specific game requirements for the model to implement
    - game_elements: list, list of game elements/objects to include in the game
    - examples: list, list of example game implementations for few-shot learning

    Returns:
    - str, the generated Pygame game code from the model
    """
    prompt = fewshot_prompt(context, question, game_elements, examples)
    result = model.invoke(prompt)

    return result

In [ ]:
examples = [
{
    "question": "Create a simple Pong game with paddle and ball movement",
    "context": "Basic Pong game with two paddles and a bouncing ball",
    "output": """
import pygame
import sys
import random

# Initialize Pygame
pygame.init()

# Constants
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
PADDLE_WIDTH = 15
PADDLE_HEIGHT = 90
BALL_SIZE = 15
PADDLE_SPEED = 5
BALL_SPEED = 7

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)

# Set up the display
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Pong Game")
clock = pygame.time.Clock()

# Game objects
player = pygame.Rect(50, WINDOW_HEIGHT//2 - PADDLE_HEIGHT//2, PADDLE_WIDTH, PADDLE_HEIGHT)
opponent = pygame.Rect(WINDOW_WIDTH - 50 - PADDLE_WIDTH, WINDOW_HEIGHT//2 - PADDLE_HEIGHT//2, PADDLE_WIDTH, PADDLE_HEIGHT)
ball = pygame.Rect(WINDOW_WIDTH//2 - BALL_SIZE//2, WINDOW_HEIGHT//2 - BALL_SIZE//2, BALL_SIZE, BALL_SIZE)

# Ball speed
ball_speed_x = BALL_SPEED * random.choice((1, -1))
ball_speed_y = BALL_SPEED * random.choice((1, -1))

# Score
player_score = 0
opponent_score = 0
game_font = pygame.font.Font(None, 32)

def ball_animation():
    global ball_speed_x, ball_speed_y, player_score, opponent_score
    
    ball.x += ball_speed_x
    ball.y += ball_speed_y
    
    if ball.top <= 0 or ball.bottom >= WINDOW_HEIGHT:
        ball_speed_y *= -1
        
    if ball.left <= 0:
        opponent_score += 1
        ball_restart()
        
    if ball.right >= WINDOW_WIDTH:
        player_score += 1
        ball_restart()
        
    if ball.colliderect(player) or ball.colliderect(opponent):
        ball_speed_x *= -1

def ball_restart():
    ball.center = (WINDOW_WIDTH//2, WINDOW_HEIGHT//2)
    ball_speed_y *= random.choice((1, -1))
    ball_speed_x *= random.choice((1, -1))

def player_animation():
    player.y += player_speed
    
    if player.top <= 0:
        player.top = 0
    if player.bottom >= WINDOW_HEIGHT:
        player.bottom = WINDOW_HEIGHT

def opponent_ai():
    if opponent.top < ball.y:
        opponent.y += PADDLE_SPEED
    if opponent.bottom > ball.y:
        opponent.y -= PADDLE_SPEED
        
    if opponent.top <= 0:
        opponent.top = 0
    if opponent.bottom >= WINDOW_HEIGHT:
        opponent.bottom = WINDOW_HEIGHT

# Game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    
    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP]:
        player_speed = -PADDLE_SPEED
    elif keys[pygame.K_DOWN]:
        player_speed = PADDLE_SPEED
    else:
        player_speed = 0
    
    ball_animation()
    player_animation()
    opponent_ai()
    
    # Drawing
    screen.fill(BLACK)
    pygame.draw.rect(screen, BLUE, player)
    pygame.draw.rect(screen, RED, opponent)
    pygame.draw.ellipse(screen, WHITE, ball)
    pygame.draw.aaline(screen, WHITE, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT))
    
    player_text = game_font.render(str(player_score), False, WHITE)
    screen.blit(player_text, (WINDOW_WIDTH//4, 20))
    
    opponent_text = game_font.render(str(opponent_score), False, WHITE)
    screen.blit(opponent_text, (3*WINDOW_WIDTH//4, 20))
    
    pygame.display.flip()
    clock.tick(60)
"""
},
{
    "question": "Create a simple Snake game with food collection and score tracking",
    "context": "Classic Snake game with growing snake and food mechanics",
    "output": """
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Constants
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
GRID_SIZE = 20
GRID_WIDTH = WINDOW_WIDTH // GRID_SIZE
GRID_HEIGHT = WINDOW_HEIGHT // GRID_SIZE

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Set up the display
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()

class Snake:
    def _init_(self):
        self.positions = [(GRID_WIDTH//2, GRID_HEIGHT//2)]
        self.direction = (1, 0)
        self.grow = False
        
    def get_head_position(self):
        return self.positions[0]
    
    def update(self):
        cur = self.get_head_position()
        x, y = self.direction
        new = ((cur[0] + x) % GRID_WIDTH, (cur[1] + y) % GRID_HEIGHT)
        
        if new in self.positions[3:]:
            return False
        
        self.positions.insert(0, new)
        if not self.grow:
            self.positions.pop()
        else:
            self.grow = False
        return True
    
    def reset(self):
        self.positions = [(GRID_WIDTH//2, GRID_HEIGHT//2)]
        self.direction = (1, 0)
        self.grow = False
    
    def render(self, surface):
        for p in self.positions:
            rect = pygame.Rect((p[0] * GRID_SIZE, p[1] * GRID_SIZE), (GRID_SIZE, GRID_SIZE))
            pygame.draw.rect(surface, GREEN, rect)

class Food:
    def _init_(self):
        self.position = (0, 0)
        self.randomize_position()
        
    def randomize_position(self):
        self.position = (random.randint(0, GRID_WIDTH-1), random.randint(0, GRID_HEIGHT-1))
    
    def render(self, surface):
        rect = pygame.Rect((self.position[0] * GRID_SIZE, self.position[1] * GRID_SIZE), (GRID_SIZE, GRID_SIZE))
        pygame.draw.rect(surface, RED, rect)

def main():
    snake = Snake()
    food = Food()
    score = 0
    font = pygame.font.Font(None, 36)
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and snake.direction != (0, 1):
                    snake.direction = (0, -1)
                elif event.key == pygame.K_DOWN and snake.direction != (0, -1):
                    snake.direction = (0, 1)
                elif event.key == pygame.K_LEFT and snake.direction != (1, 0):
                    snake.direction = (-1, 0)
                elif event.key == pygame.K_RIGHT and snake.direction != (-1, 0):
                    snake.direction = (1, 0)
        
        if not snake.update():
            snake.reset()
            food.randomize_position()
            score = 0
            continue
        
        if snake.get_head_position() == food.position:
            snake.grow = True
            food.randomize_position()
            score += 1
        
        screen.fill(BLACK)
        snake.render(screen)
        food.render(screen)
        
        score_text = font.render(f'Score: {score}', True, WHITE)
        screen.blit(score_text, (10, 10))
        
        pygame.display.update()
        clock.tick(10)

if _name_ == '_main_':
    main()
"""
},
{
    "question": "Create a simple Breakout-style game with paddle, ball, and bricks",
    "context": "Breakout game with paddle movement, ball physics, and brick destruction",
    "output": """
import pygame
import sys
import random

# Initialize Pygame
pygame.init()

# Constants
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
PADDLE_WIDTH = 100
PADDLE_HEIGHT = 20
BALL_SIZE = 10
BRICK_WIDTH = 80
BRICK_HEIGHT = 30
BRICK_ROWS = 5
BRICK_COLS = 10

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)
ORANGE = (255, 165, 0)

# Set up the display
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Breakout Game")
clock = pygame.time.Clock()

class Paddle:
    def _init_(self):
        self.rect = pygame.Rect(WINDOW_WIDTH//2 - PADDLE_WIDTH//2, WINDOW_HEIGHT - 40, PADDLE_WIDTH, PADDLE_HEIGHT)
        self.speed = 8
        
    def move(self, direction):
        if direction == 'left' and self.rect.left > 0:
            self.rect.x -= self.speed
        if direction == 'right' and self.rect.right < WINDOW_WIDTH:
            self.rect.x += self.speed
    
    def draw(self):
        pygame.draw.rect(screen, BLUE, self.rect)

class Ball:
    def _init_(self):
        self.rect = pygame.Rect(WINDOW_WIDTH//2 - BALL_SIZE//2, WINDOW_HEIGHT - 60, BALL_SIZE, BALL_SIZE)
        self.speed_x = 5
        self.speed_y = -5
        
    def move(self):
        self.rect.x += self.speed_x
        self.rect.y += self.speed_y
        
        # Wall collision
        if self.rect.left <= 0 or self.rect.right >= WINDOW_WIDTH:
            self.speed_x *= -1
        if self.rect.top <= 0:
            self.speed_y *= -1
    
    def draw(self):
        pygame.draw.rect(screen, WHITE, self.rect)
    
    def reset(self):
        self.rect.center = (WINDOW_WIDTH//2, WINDOW_HEIGHT - 60)
        self.speed_x = 5 * random.choice((1, -1))
        self.speed_y = -5

class Brick:
    def _init_(self, x, y, color):
        self.rect = pygame.Rect(x, y, BRICK_WIDTH, BRICK_HEIGHT)
        self.color = color
        self.active = True
    
    def draw(self):
        if self.active:
            pygame.draw.rect(screen, self.color, self.rect)

def create_bricks():
    bricks = []
    colors = [RED, ORANGE, YELLOW, GREEN, BLUE]
    
    for row in range(BRICK_ROWS):
        for col in range(BRICK_COLS):
            x = col * (BRICK_WIDTH + 2) + 1
            y = row * (BRICK_HEIGHT + 2) + 50
            brick = Brick(x, y, colors[row])
            bricks.append(brick)
    
    return bricks

def check_collision(ball, paddle, bricks):
    # Paddle collision
    if ball.rect.colliderect(paddle.rect):
        ball.speed_y *= -1
        # Adjust ball direction based on where it hits the paddle
        relative_intersect_x = (paddle.rect.centerx - ball.rect.centerx) / (PADDLE_WIDTH / 2)
        ball.speed_x = -relative_intersect_x * 7
    
    # Brick collision
    for brick in bricks:
        if brick.active and ball.rect.colliderect(brick.rect):
            brick.active = False
            ball.speed_y *= -1
            return True
    
    return False

def main():
    paddle = Paddle()
    ball = Ball()
    bricks = create_bricks()
    score = 0
    lives = 3
    font = pygame.font.Font(None, 36)
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            paddle.move('left')
        if keys[pygame.K_RIGHT]:
            paddle.move('right')
        
        ball.move()
        
        # Check for ball falling off screen
        if ball.rect.bottom >= WINDOW_HEIGHT:
            lives -= 1
            ball.reset()
            if lives <= 0:
                # Game over
                lives = 3
                score = 0
                bricks = create_bricks()
        
        # Check collisions
        if check_collision(ball, paddle, bricks):
            score += 10
        
        # Check if all bricks are destroyed
        if all(not brick.active for brick in bricks):
            bricks = create_bricks()
            ball.reset()
        
        # Drawing
        screen.fill(BLACK)
        paddle.draw()
        ball.draw()
        
        for brick in bricks:
            brick.draw()
        
        # Display score and lives
        score_text = font.render(f'Score: {score}', True, WHITE)
        lives_text = font.render(f'Lives: {lives}', True, WHITE)
        screen.blit(score_text, (10, 10))
        screen.blit(lives_text, (WINDOW_WIDTH - 100, 10))
        
        pygame.display.flip()
        clock.tick(60)

if _name_ == '_main_':
    main()
"""
},
{
    "question": "Create a simple Space Invaders-style game with player ship and enemies",
    "context": "Space Invaders game with player movement, enemy formation, and shooting mechanics",
    "output": """
import pygame
import sys
import random

# Initialize Pygame
pygame.init()

# Constants
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
PLAYER_WIDTH = 50
PLAYER_HEIGHT = 30
ENEMY_WIDTH = 40
ENEMY_HEIGHT = 30
BULLET_WIDTH = 4
BULLET_HEIGHT = 10
ENEMY_ROWS = 5
ENEMY_COLS = 8

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# Set up the display
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Space Invaders")
clock = pygame.time.Clock()

class Player:
    def _init_(self):
        self.rect = pygame.Rect(WINDOW_WIDTH//2 - PLAYER_WIDTH//2, WINDOW_HEIGHT - 50, PLAYER_WIDTH, PLAYER_HEIGHT)
        self.speed = 5
        
    def move(self, direction):
        if direction == 'left' and self.rect.left > 0:
            self.rect.x -= self.speed
        if direction == 'right' and self.rect.right < WINDOW_WIDTH:
            self.rect.x += self.speed
    
    def draw(self):
        pygame.draw.rect(screen, GREEN, self.rect)

class Enemy:
    def _init_(self, x, y):
        self.rect = pygame.Rect(x, y, ENEMY_WIDTH, ENEMY_HEIGHT)
        self.direction = 1
        self.speed = 1
        
    def move(self):
        self.rect.x += self.speed * self.direction
        
    def drop_and_reverse(self):
        self.rect.y += 20
        self.direction *= -1
    
    def draw(self):
        pygame.draw.rect(screen, RED, self.rect)

class Bullet:
    def _init_(self, x, y, direction):
        self.rect = pygame.Rect(x, y, BULLET_WIDTH, BULLET_HEIGHT)
        self.speed = 7
        self.direction = direction  # 1 for player bullet, -1 for enemy bullet
        
    def move(self):
        self.rect.y -= self.speed * self.direction
        
    def draw(self):
        pygame.draw.rect(screen, WHITE, self.rect)

def create_enemies():
    enemies = []
    for row in range(ENEMY_ROWS):
        for col in range(ENEMY_COLS):
            x = col * (ENEMY_WIDTH + 10) + 50
            y = row * (ENEMY_HEIGHT + 10) + 50
            enemies.append(Enemy(x, y))
    return enemies

def main():
    player = Player()
    enemies = create_enemies()
    bullets = []
    score = 0
    lives = 3
    font = pygame.font.Font(None, 36)
    enemy_move_counter = 0
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    # Fire bullet
                    bullet = Bullet(player.rect.centerx - BULLET_WIDTH//2, player.rect.top, 1)
                    bullets.append(bullet)
        
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            player.move('left')
        if keys[pygame.K_RIGHT]:
            player.move('right')
        
        # Move enemies
        enemy_move_counter += 1
        if enemy_move_counter >= 30:
            enemy_move_counter = 0
            should_drop = False
            
            for enemy in enemies:
                enemy.move()
                if enemy.rect.left <= 0 or enemy.rect.right >= WINDOW_WIDTH:
                    should_drop = True
            
            if should_drop:
                for enemy in enemies:
                    enemy.drop_and_reverse()
        
        # Move bullets
        for bullet in bullets[:]:
            bullet.move()
            if bullet.rect.bottom < 0 or bullet.rect.top > WINDOW_HEIGHT:
                bullets.remove(bullet)
        
        # Check bullet collisions
        for bullet in bullets[:]:
            if bullet.direction == 1:  # Player bullet
                for enemy in enemies[:]:
                    if bullet.rect.colliderect(enemy.rect):
                        enemies.remove(enemy)
                        bullets.remove(bullet)
                        score += 10
                        break
            else:  # Enemy bullet
                if bullet.rect.colliderect(player.rect):
                    bullets.remove(bullet)
                    lives -= 1
                    if lives <= 0:
                        # Game over
                        lives = 3
                        score = 0
                        enemies = create_enemies()
                        bullets.clear()
        
        # Enemy shooting (random)
        if random.randint(1, 100) == 1 and enemies:
            shooter = random.choice(enemies)
            bullet = Bullet(shooter.rect.centerx - BULLET_WIDTH//2, shooter.rect.bottom, -1)
            bullets.append(bullet)
        
        # Check if enemies reached bottom
        for enemy in enemies:
            if enemy.rect.bottom >= WINDOW_HEIGHT - 100:
                lives = 0
        
        # Check if all enemies destroyed
        if not enemies:
            enemies = create_enemies()
        
        # Drawing
        screen.fill(BLACK)
        player.draw()
        
        for enemy in enemies:
            enemy.draw()
        
        for bullet in bullets:
            bullet.draw()
        
        # Display score and lives
        score_text = font.render(f'Score: {score}', True, WHITE)
        lives_text = font.render(f'Lives: {lives}', True, WHITE)
        screen.blit(score_text, (10, 10))
        screen.blit(lives_text, (WINDOW_WIDTH - 100, 10))
        
        pygame.display.flip()
        clock.tick(60)

if _name_ == '_main_':
    main()
"""
}
]


In [ ]:
# Example usage for game development
context = "Create an interactive Breakout-style game with paddle, ball, and brick mechanics"
question = "Develop a complete Breakout game where the player controls a paddle to bounce a ball and destroy bricks. Include score tracking, multiple lives, and different colored bricks."
game_elements = ["paddle", "ball", "bricks", "score", "lives", "collision detection"]

# Generate and display the game code
result = get_answer_using_fewshot(context, question, game_elements, examples)
print(f"Generated Game Code:\n{result}")